<h1><b>Statistique en Bioinformatique : </b> TME 5 et 6 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME5_6.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre via moodle </li>
</div>
</div>

Nom etudiant 1 : Grichi Ihsân
<br>
Nom etudiant 2 :
<br>

<h3>Introduction</h3>
Un casino parfois malhonnête (occasionally dishonest casino) utilise 2 types de pieces : fair et unfair. <br>
La matrice de transition entre les états cachés est:<br>
${\cal S}=\{F,U\}$ (fair, unfair):
$$
p = \left(
\begin{array}{cc}
0.99 & 0.01\\
0.05 & 0.95
\end{array}
\right)\ ,
$$

les probabilités d'émission des symboles 
${\cal O} = \{H,T\}$ (head, tail):
\begin{eqnarray}
e_F(H) =  0.5 &\ \ \ \ &
e_F(T) = 0.5 \nonumber\\
e_U(H) = 0.9 &\ \ \ \ &
e_U(T) = 0.1 \nonumber
\end{eqnarray}

<br> Et la condition initiale $\pi^{(0)} = (0.999,0.001)$ (le jeux commence presque toujours avec le pieces juste (fair).

<b>Exercice 1</b>:
<u>Simulation</u>: Écrire une fonction qui simule $T$ jets de pièces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

#states
S = { 0:'F',1 :'U'}

#transition probability matrix
Pij = np.array([[0.99,0.01], [0.05,0.95]])

#emision symbols 
O = {0:'H', 1: 'T'}

#emission probability matrix
Ei = np.array([[0.5,0.5], [0.9,0.1]]) #ça aurait dû être Eio

# initial Condition
pi0=np.array([0.999,0.001])

#number of jets
T = 2000

In [2]:
import random as rd

# Fonction qui simule T jets de pieces
def jets(T, pi0, Eij, Pij):
    """
      simulation of occasionally dishonest casino
      input1 T: number of jets
      input2 pi0: initial condition
      input3 Eij: emission probability matrix
      input4 Pij: transition probability matrix
      output1 jetsRes: matrix |2xT| containing simulations
    """
    # Creation du tableau
    jetsRes = np.zeros((T,len(pi0)),dtype=int)
    
    
    PI0 = pi0.copy()
    PI0 = np.cumsum(PI0)
    
    EIJ = Eij.copy()
    EIJ = np.cumsum(EIJ, axis = 1)
    
    PIJ = Pij.copy()
    PIJ = np.cumsum(PIJ, axis = 1) 
    
    jet1 = rd.random()
    etat = 0
    
    jet2 = rd.random()
    obs = 0
    
    while jet1 >= PI0[etat]:
        etat+=1
    
    while jet2 >= EIJ[etat, obs]:
        obs+=1
    
    jetsRes[0] = [etat, obs]
    
    for t in range(1, T):
        
        jet1 = rd.random()
        i = 0
        
        while jet1 >= PIJ[etat, i]:
            i+=1
        
        etat = i
        
        jet2 = rd.random()
        j = 0
        
        while jet2 >= EIJ[etat, j]:
            j+=1
            
        obs = j
        
        jetsRes[t] = [etat, obs]
    return jetsRes



def printSimulation(resultat):
    for i in resultat : 
        print (S[i[0]], O[i[1]])

jetsRes = jets(T, pi0, Ei, Pij)
#printSimulation(jetsRes)

<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Écrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ainsi que sa probabilité. Pour tester votre fonction utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation).


In [18]:
# Algorithme de Viterbi
import operator

def viterbi(jets, Pij, Ei, pi0, nS, nO, enLog):
    """
    Implement Viterbi algorithm
    input1 jets: matrix |2xT| containing simulations
    input4 Pij: transition probability matrix
    input3 Ei: emission probability matrix
    input4 pi0: initial condition
    input5 nS: number of states
    input6 nO: number of observations
    input7 enLog: bool, when True we apply log to avoid underflow
    output1 i_star: most problable path
    output2 prob: probability associated to the most problable path
    """
    
    obs = jets[:,1]
    T = len(obs) #Nombre d'observations (longueur des observations)
    logPij = np.log(Pij)
    logpi0 = np.log(pi0)
    logEi = np.log(Ei)
    i_star = np.zeros((T))
    prob = 1
    
    v = np.zeros((T, nS))
    for i in range(nO):
        if enLog :
            v[0, i] = logpi0[i] + logEi[i, obs[0]]
        else :
            v[0, i] = pi0[i] * Ei[i, obs[0]]
        
    for i in range(1, T):
        if enLog:
            v[i, 0] = logEi[0, obs[i]] + max ( v[i-1, 0] + logPij[0, 0], v[i-1, 1] + logPij[1, 0] )
        else :
            v[i, 0] = Ei[0, obs[i]] * max ( v[i-1, 0] * Pij[0, 0], v[i-1, 1] * Pij[1, 0] )
        if enLog:
            v[i, 1] = logEi[1, obs[i]] + max( v[i-1, 1] + logPij[1, 1], v[i-1, 0] + logPij[0, 1] )
        else:
            v[i, 1] = Ei[1, obs[i]] * max( v[i-1, 1] * Pij[1, 1], v[i-1, 0] * Pij[0, 1] )
    
    
    i_star[-1] = int(np.argmax(v[-1]))
    for i in range(T-2, 0):
        if enLog:
            line = [v[i, 0] + logPij[0, int(i_star[i+1]) ], v[i, 1] + logPij[1, int(i_star[i+1]) ] ]
        else:
            line = [v[i, 0] * Pij[0, int(i_star[i+1]) ], v[i, 1] * Pij[1, int(i_star[i+1]) ] ]
        
        i_star[i] = np.argmax(line)
        
    return i_star, max([v[-1]])

def analyseResultats(jets, estimation):    
    """
    Compare expected and obtained paths
    input1 jets: expected path
    input2 estimation: obtained path
    output1 error: percentage error
    """
    error = np.mean(np.abs(jets[:, 0] - estimation)) * 100
    return error

i_est, P_est = viterbi(jetsRes, Pij, Ei, pi0, 2, 2, False)
error = analyseResultats(jetsRes, i_est)
print('erreur d\'estimation de viterbi:')
print(error,'%')
#print('Probabilité estimé:')
#print(P_est)

[  0. -inf]
erreur d'estimation de viterbi:
15.0 %


<ipython-input-18-46a19c374954>:21: RuntimeWarning: divide by zero encountered in log
  logpi0 = np.log(pi0)


<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Écrire une fonction qu'utilise tous les résultats de la simulation
(états et symboles) pour compter les nombres d'occurrence $N_{ij}$ est $M_{iO}$ définis en cours. Estimer $P_{ij}$ est $E_i(O)$. Attention, pour éviter les probabilités à zéro nous allons utiliser les pseudo-count.

In [19]:
# Estimation de Parametres par contage
def nombresOccurrence(jets, nS, nO):
    """
    Parameter estimation
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    output1 Nij: transition probability matrix
    output2 Mio: emission probability matrix
    output3 pi0: initial condition 
    """

    Nij = np.ones((nS,nS)) #pseudo-count = 1
    Mio = np.ones((nS,nO))  #pseudo-count = 1
    pi0 = np.zeros((nS))
    n = len(jets[:, 0])
    
    state = jets[0, 0]
    obs = jets[0, 1]
    Mio[state, obs] += 1
    pi0[state] = 1
    
    
    for i in range(1, n):
        Nij[state, jets[i, 0]] += 1
        Mio[state, jets[i, 1]] += 1
        state = jets[i, 0]
    
    Nij /= np.sum(Nij, axis = 0)
    Mio /= np.sum(Mio, axis = 0)
    
    return Nij.T, Mio.T, pi0

Nij, Mio, pi0 = nombresOccurrence(jetsRes, 2, 2)

print('Pij estimé:')
print(Nij)
print('\nEio estimé:')
print(Mio)
print('\npi0 estimé:')
print(pi0)

Pij estimé:
[[0.98765432 0.01234568]
 [0.07284768 0.92715232]]

Eio estimé:
[[0.77200704 0.22799296]
 [0.9516129  0.0483871 ]]

pi0 estimé:
[1. 0.]


3.2) <u> Viterbi training </u>: Écrire une fonction qui utilise
seulement la séquence $(O_t)_{t=0:T}$ (2emme colonne de la simulation) pour estimer les
paramètres $P_{ij}$ est $Ei(O)$. On s’arrêtera quand les différences entre les logVraissamblance est inférieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres aléatoire).


In [20]:
import matplotlib.pyplot as plt
import random as rd

# Initialisation aleatoire de Pij, Eij, pi0
def InititRandom(nS, nO):
    """
    randomly initialisations 
    input1 nS: number of states
    input2 nO: number of observations
    output1 Pij_init: transition probability matrix
    output2 Ei_init: emission probability matrix
    output3 pi0_init: initial condition 
    """
    
    rd.seed(10)
    
    Pij_init = np.zeros((nS, nS))
    
    for i in range(nS):
        for j in range(nS):
            jet = rd.random()
            Pij_init[i, j] = jet
            
    Pij_init /= np.sum(Pij_init, axis = 1).reshape(nS, 1)
    
    Ei_init = np.zeros((nO, nS))
    
    for i in range(nO):
        for j in range(nS):
            jet = rd.random()
            Ei_init[i, j] = jet
            
    Ei_init /= np.sum(Ei_init, axis = 1).reshape(nS, 1)
    
    pi0_init = np.zeros((1, nS))
    
    for j in range(nS):
        jet = rd.random()
        pi0_init[0, j] = jet
        
    pi0_init /= np.sum(pi0_init)
    
    return Pij_init, Ei_init, pi0_init

# Calcule log Vraissemblance
def logLikelhihood(Aij,Bij,pi0,jets):
    """
    compute Log Likelihood 
    input1 Pij: transition probability matrix
    input2 Ei: emission probability matrix
    input3 pi0: initial condition 
    input4 jets: matrix |2xT| containing data
    """
    etat_seq = jets[:,0]
    obs_seq = jets[:,1]
    T = len(obs_seq) #Nombre d'observations (longueur des observations)
    lLikelihood = np.log(pi0[etat_seq[0]]) + np.log(Bij[etat_seq[0], obs_seq[0]]) 
    for t in range(1, T):
        lLikelihood += np.log(Aij[etat_seq[t-1], etat_seq[t]]) + np.log(Bij[etat_seq[t], obs_seq[t]])
    return lLikelihood


def Training(jets, nS, nO):
    """
    Viterbi Training
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    output1 Pij_est: transition probability matrix
    output2 Ei_est: emission probability matrix
    output3 pi0_est: initial condition 
    output4 lLikelihood: log Likelihood
    """
    jets_est = np.array(jets)
    Pij_est, Ei_est, pi0_est = InititRandom(nS, nO)
    
    nIteration = 10000
    criterion = 1e-04
    lLikelihood = np.zeros((nIteration))
    t = 1
    stop = False
    while t < nIteration and stop == False:
        i_star, prob = viterbi(jets, Pij_est, Ei_est, pi0_est[0], nS, nO, False)
        jets_est[:, 0] = i_star
        Pij_est, Ei_est, pi0_est = nombresOccurrence(jets_est, nS, nO)
        lLikelihood[t] = logLikelhihood(Pij_est, Ei_est, pi0_est, jets_est)
        if np.abs(lLikelihood[t-1] - lLikelihood[t]) < criterion:
            stop = True
        t+1  
        
    return Pij_est, Ei_est, pi0_est, lLikelihood
    
#imprimer les Parametres du Viterbi Training
Pij_est, Ei_est, pi0_est, lLikelihood = Training(jetsRes, 2, 2)
itCount = len(lLikelihood)
print('Le modèle est convergé après '+str(itCount)+' itérations.')
print('\nPij estimée:')
print(Pij_est)
print('\nEij estimée:')
print(Ei_est)

[-0.48828671 -0.95108121]
0.0


IndexError: invalid index to scalar variable.

<font color="blue">
Remark:
</font>

3.3) <u>Viterbi training deuxième version</u>. 
<BR>Écrivez une version de 3.2 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire desparamètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcul la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de lalog-vraisemblance.

Qu'est-ce que vous observez?



In [21]:
# Viterbi Training  deuxiemme version

def TrainingV2(jets, nS, nO, nIterat=100):
    """
    Viterbi Training version 2.0
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    input4 nIterat: number of iterations
    output1 Pij_best: best transition probability matrix
    output2 Ei_best: best emission probability matrix
    output3 pi0_best: best initial condition 
    output4 lLikelihood_best: best log Likelihood
    """

    Pij_best = []
    Ei_best = []
    pi0_best = []
    lLikelihood_best = -10000
    

    return Pij_best, Ei_best, pi0_best, lLikelihood_best
    

# Imprimer les Parametres du Viterbi Training deuxiemme version
nIterat = 100
Pij_best, Ei_best, pi0_best, lLikelihood_best = TrainingV2(jetsRes, 2, 2, nIterat)

print('Meilleur Pij estimée:');
print(Pij_best)
print('\nMeilleur Eij estimée:')
print(Ei_best)

Meilleur Pij estimée:
[]

Meilleur Eij estimée:
[]


<font color="blue">
Remark:
</font>